<a href="https://colab.research.google.com/github/nkampel/data_science_tutorial/blob/main/Profiling_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the necessary libraries
!pip install transformers
!pip install braindecode

In [56]:
import torch
import torch.nn as nn
from transformers import GPT2Model, GPT2Config
from braindecode.models import EEGInceptionMI

class EncoderEEGModel(nn.Module):
    def __init__(self, n_chans, n_outputs, input_window_seconds, sfreq):
        super(EncoderEEGModel, self).__init__()
        # Define your EEG encoder model here
        self.eeg_model = EEGInceptionMI(n_chans, n_outputs, input_window_seconds, sfreq, add_log_softmax=False)

    def forward(self, eeg_data):
        # Forward pass of EEG encoder
        eeg_output = self.eeg_model(eeg_data)
        return eeg_output

class CombinedModel(nn.Module):
    def __init__(self, eeg_model, gpt2_model):
        super(CombinedModel, self).__init__()
        self.eeg_model = eeg_model
        self.gpt2_model = gpt2_model

    def forward(self, eeg_data):
        # Forward pass of EEG model
        eeg_output = self.eeg_model(eeg_data)
        # Forward pass of GPT-2 model
        gpt2_output = self.gpt2_model(inputs_embeds=eeg_output)
        return gpt2_output



# Initialize your EEG model
n_chans = 22
n_outputs = 768
input_window_seconds = 4.5
sfreq = 250
n_times = int(4.5*250)


eeg_model = EncoderEEGModel(n_chans, n_outputs, input_window_seconds, sfreq)

# Initialize GPT-2 model
gpt2_config = GPT2Config.from_pretrained("gpt2")
gpt2_model = GPT2Model(gpt2_config)

# Initialize combined model
combined_model = CombinedModel(eeg_model, gpt2_model)

# Example usage:
eeg_data = torch.randn(1, n_chans, n_times)  # Example EEG data, shape: [batch_size, n_chans, n_times]
output = combined_model(eeg_data)



RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)

In [73]:
import torch
import torch.nn as nn
from transformers import GPT2Config, GPT2Model
from braindecode.models import EEGInceptionMI

class EncoderEEGModel(nn.Module):
    def __init__(self, n_chans, n_outputs, input_window_seconds, sfreq):
        super(EncoderEEGModel, self).__init__()
        # Define your EEG encoder model here
        self.eeg_model = EEGInceptionMI(n_chans, n_outputs, input_window_seconds, sfreq, add_log_softmax=False)

    def forward(self, eeg_data):
        # Forward pass of EEG encoder
        eeg_output = self.eeg_model(eeg_data)
        return eeg_output

class CombinedModel(nn.Module):
    def __init__(self, eeg_model, gpt2_embedding_size, n_gpt2_blocks=12):
        super(CombinedModel, self).__init__()
        self.eeg_model = eeg_model
        self.gpt2_model = self.initialize_gpt2_model(gpt2_embedding_size, n_gpt2_blocks)
        self.embedding_size = gpt2_embedding_size

    def initialize_gpt2_model(self, gpt2_embedding_size, n_gpt2_blocks):
        # Initialize GPT-2 model from scratch
        gpt2_config = GPT2Config(n_layer=n_gpt2_blocks, n_embd=gpt2_embedding_size)
        gpt2_model = GPT2Model(gpt2_config)
        return gpt2_model

    def forward(self, eeg_data):
        # Forward pass of EEG model
        eeg_output = self.eeg_model(eeg_data)

        # Resize EEG output to match the embedding size of GPT-2
        eeg_output_resized = eeg_output.unsqueeze(1).expand(-1, eeg_output.size(1), self.embedding_size)

        # Forward pass of GPT-2 model
        gpt2_output = self.gpt2_model(inputs_embeds=eeg_output_resized)
        return gpt2_output

# Example usage:
n_chans = 22
n_outputs = 768
input_window_seconds = 4.5
sfreq = 250
n_times = int(4.5 * 250)
n_gpt2_blocks = 6
gpt2_embedding_size = 768  # Match the embedding size of the GPT-2 model

eeg_model = EncoderEEGModel(n_chans, n_outputs, input_window_seconds, sfreq)
combined_model = CombinedModel(eeg_model, gpt2_embedding_size, n_gpt2_blocks)

# Example usage:
eeg_data = torch.randn(1, n_chans, n_times)  # Example EEG data, shape: [batch_size, n_chans, n_times]
output = combined_model(eeg_data)


RuntimeError: The expanded size of the tensor (384) must match the existing size (768) at non-singleton dimension 2.  Target sizes: [-1, 768, 384].  Tensor sizes: [1, 1, 768]

In [68]:
combined_model

CombinedModel(
  (eeg_model): EncoderEEGModel(
    (eeg_model): EEGInceptionMI(
      (activation): ReLU()
      (ensuredims): Ensure4d()
      (dimshuffle): Rearrange('batch C T 1 -> batch C 1 T')
      (initial_inception_module): _InceptionModuleMI(
        (bottleneck): Conv2d(22, 48, kernel_size=(1, 1), stride=(1, 1))
        (pooling): MaxPool2d(kernel_size=(1, 25), stride=1, padding=(0, 12), dilation=1, ceil_mode=False)
        (pooling_conv): Conv2d(22, 48, kernel_size=(1, 1), stride=(1, 1))
        (conv_list): ModuleList(
          (0): Conv2d(48, 48, kernel_size=(1, 25), stride=(1, 1), padding=same)
          (1): Conv2d(48, 48, kernel_size=(1, 75), stride=(1, 1), padding=same)
          (2): Conv2d(48, 48, kernel_size=(1, 125), stride=(1, 1), padding=same)
          (3): Conv2d(48, 48, kernel_size=(1, 175), stride=(1, 1), padding=same)
          (4): Conv2d(48, 48, kernel_size=(1, 225), stride=(1, 1), padding=same)
        )
        (bn): BatchNorm2d(288, eps=1e-05, momentum

In [65]:
# Example usage:
eeg_data = torch.randn(1, n_chans, n_times)  # Example EEG data, shape: [batch_size, n_chans, n_times]
output = combined_model(eeg_data)

# Get the output from the EEG model
eeg_output = eeg_model(eeg_data)

# Manually embed the output of the EEG model
# You may need to adjust this step based on the specific requirements of your GPT-2 model
# Here, we assume that the EEG output can be directly used as input embeddings for the GPT-2 model
embedded_output = eeg_output

# Forward pass of GPT-2 model with the embedded EEG output
gpt_output = gpt2_model(inputs_embeds=embedded_output)

outputs_different = not torch.allclose(eeg_output[0], gpt_output[0])

if outputs_different:
    print("The outputs of the EEG model and the GPT-2 model are different.")
else:
    print("The outputs of the EEG model and the GPT-2 model are the same.")

The outputs of the EEG model and the GPT-2 model are different.


In [60]:
gpt2_output[0].shape

torch.Size([1, 768])

In [46]:
# Example usage:
eeg_data = torch.randn(1, n_chans, n_times)  # Example EEG data, shape: [batch_size, n_chans, n_times]
output = combined_model(eeg_data)

# Print the shapes of individual tensors in the output tuple
for i, tensor in enumerate(output[1]):
    print(f"GPT-2 Model Output Tensor {i} Shape:", tensor.shape)

AttributeError: 'tuple' object has no attribute 'shape'

In [43]:
# Initialize your EEG model
n_chans = 22
n_outputs = 768
input_window_seconds = 4.5
sfreq = 250
n_times = int(4.5*250)


eeg_model = EncoderEEGModel(n_chans, n_outputs, input_window_seconds, sfreq)

# Initialize GPT-2 model
gpt2_config = GPT2Config.from_pretrained("gpt2")
gpt2_model = GPT2Model(gpt2_config)

# Initialize combined model
combined_model = CombinedModel(eeg_model, gpt2_model)

# Example usage:
eeg_data = torch.randn(1, n_chans, n_times)  # Example EEG data, shape: [batch_size, n_chans, n_times]
output = combined_model(eeg_data)

print(f"GPT-2 Model Output Tensor Shape:", output[0].shape)

GPT-2 Model Output Tensor Shape: torch.Size([1, 768])


In [37]:
for i, tensor in enumerate(output[1]):
    print(f"GPT-2 Model Output Tensor {i} Shape:", tensor.shape)

AttributeError: 'tuple' object has no attribute 'shape'

torch.Size([1, 768])

In [26]:
from torchviz import make_dot

# Visualize EEG model
eeg_output = eeg_model(torch.randn(1, n_chans, n_times))
make_dot(eeg_output, params=dict(eeg_model.named_parameters())).render("eeg_model", format="png")

# Visualize GPT-2 model with EEG embeddings
gpt2_output = gpt2_model(inputs_embeds=eeg_output)
make_dot(gpt2_output, params=dict(gpt2_model.named_parameters())).render("gpt2_model", format="png")


TypeError: unhashable type: 'BaseModelOutputWithPastAndCrossAttentions'

In [27]:
eeg_output

tensor([[-2.5136e-01, -1.2143e+00, -4.8543e-01, -4.8605e-01,  9.9775e-02,
         -8.2560e-01,  3.3995e-01, -4.9248e-01,  1.3174e-01,  5.9843e-01,
          4.3467e-01, -3.3816e-01, -6.4134e-01, -3.0705e-02, -1.4262e-01,
          1.0825e-01,  8.6665e-02, -2.1842e-01, -2.3456e-01,  3.0020e-01,
          2.2166e-01,  6.6923e-01, -3.1276e-01, -2.2700e-01, -2.0324e-01,
         -6.9813e-01,  1.4384e-01, -1.5111e-01,  3.4685e-01, -3.2506e-01,
          8.0895e-01, -1.1485e+00,  4.3543e-01, -6.8292e-01, -3.5660e-01,
          1.7431e-01, -7.9292e-01,  8.1750e-02, -5.0028e-01,  9.1045e-01,
         -6.4724e-02, -4.9543e-01, -5.5367e-02,  6.6174e-01,  3.8639e-02,
          1.6639e-01, -1.3916e-01, -2.2760e-02, -4.1277e-02, -3.0376e-02,
         -9.7262e-03, -5.2796e-01, -4.1267e-01, -6.0717e-01,  7.2121e-02,
         -9.0183e-02,  6.0659e-03,  8.6592e-01, -5.7494e-01,  2.5092e-01,
          8.9376e-02, -4.8503e-02, -4.7481e-01, -7.0623e-02, -1.4292e-01,
          7.5085e-02, -3.4870e-01,  1.

In [66]:
combined_model

CombinedModel(
  (eeg_model): EncoderEEGModel(
    (eeg_model): EEGInceptionMI(
      (activation): ReLU()
      (ensuredims): Ensure4d()
      (dimshuffle): Rearrange('batch C T 1 -> batch C 1 T')
      (initial_inception_module): _InceptionModuleMI(
        (bottleneck): Conv2d(22, 48, kernel_size=(1, 1), stride=(1, 1))
        (pooling): MaxPool2d(kernel_size=(1, 25), stride=1, padding=(0, 12), dilation=1, ceil_mode=False)
        (pooling_conv): Conv2d(22, 48, kernel_size=(1, 1), stride=(1, 1))
        (conv_list): ModuleList(
          (0): Conv2d(48, 48, kernel_size=(1, 25), stride=(1, 1), padding=same)
          (1): Conv2d(48, 48, kernel_size=(1, 75), stride=(1, 1), padding=same)
          (2): Conv2d(48, 48, kernel_size=(1, 125), stride=(1, 1), padding=same)
          (3): Conv2d(48, 48, kernel_size=(1, 175), stride=(1, 1), padding=same)
          (4): Conv2d(48, 48, kernel_size=(1, 225), stride=(1, 1), padding=same)
        )
        (bn): BatchNorm2d(288, eps=1e-05, momentum